## Xview Dataset clipping
This is a component designed to clip the dataset provided by xview. 

In [ ]:
#!pip install Pillow claimed aiobotocore botocore s3fs

In [ ]:

import os
from PIL import Image
from c3 import operator_utils
import s3fs
import pathlib
import math
from PIL import Image, TiffTags, TiffImagePlugin
import imageio


In [ ]:

#source is the path to the folder with the unzipped .tif images from xview dataset 
source = os.environ.get("directory_path")

#destination is the path to the folder which saves all the extracted tiles. 
destination = os.environ.get("destination")

#Each image is cropped using a rectangular window with edge_length tile_size_x and tile_size_y which has to be given in number of pixels 
tile_size_x = int(os.environ.get("tile_size_x", 64))
tile_size_y = int(os.environ.get("tile_size_y", 64))

#stride_x is the length in pixels the sliding window is moved to the right after each step
#For tumbling window stride_x must equal tile_size_x and stride_y must equal tile_size_y
stride_x = int(os.environ.get("stride_x", 32))
#stride_y is the length in pixels the sliding window is moved down after completing a row
stride_y = int(os.environ.get("stride_y", 32))


In [ ]:

(access_key_id_source, secret_access_key_source, endpoint_source, source) = operator_utils.explode_connection_string(source)
(access_key_id_destination, secret_access_key_destination, endpoint_destination, destination) = operator_utils.explode_connection_string(destination)


In [ ]:

s3source = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id_source,
    secret=secret_access_key_source,
    client_kwargs={'endpoint_url': endpoint_source}
)


In [ ]:

s3destination = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id_destination,
    secret=secret_access_key_destination,
    client_kwargs={'endpoint_url': endpoint_destination}
)


In [ ]:

for item in s3source.ls(source):
    s3source.get(item, item)
    _, file_extension = os.path.splitext(item)
    clipped_item = os.path.join(destination, item)

    image = Image.open(item) 
    width, height = image.size

    x_range = [0]
    while(x_range[-1] + stride_x + tile_size_x < width):
        x_range += [x_range[-1] + stride_x]
    
    y_range = [0]
    while(y_range[-1] + stride_y + tile_size_y < height):
        y_range += [y_range[-1] + stride_y]

    og_metadata = image.tag_v2
    initial_lon = og_metadata[33922][3]
    initial_lat = og_metadata[33922][4]
    pixel_height_deg = og_metadata[33550][1]
    pixel_width_deg = og_metadata[33550][0]

    for x in x_range:
        for y in y_range:
            left = x * tile_size_x
            upper = y * tile_size_y
            right = left + tile_size_x
            lower = upper + tile_size_y

            upper_left_lat = initial_lat + y  * pixel_height_deg
            upper_left_lon = initial_lon + x  * pixel_width_deg

            cropped = image.crop((x,y, x+tile_size_x, y+tile_size_y))
            
            cropped.save("temporary_exchange.tif")
            cropped = Image.open("temporary_exchange.tif")

            available_metadata = cropped.tag_v2
            
            tiffinfo = TiffImagePlugin.ImageFileDirectory_v2()
            
            for tag, value in og_metadata.items():
                if not(tag in available_metadata or tag == 339 or tag == 33922):
                    tiffinfo[tag] = value
               
            tiffinfo[33922] = (0.0,0.0,0.0, upper_left_lon, upper_left_lat, 0.0)

            dest_path = f'{clipped_item}.{x}.{y}{file_extension}'
            pathlib.Path(os.path.dirname(dest_path)).mkdir(parents=True, exist_ok=True)
            
            cropped.save(dest_path, tiffinfo = tiffinfo, save_all=True, compression="tiff_deflate")
            
            clipped_item_upload = os.path.join(destination, os.path.basename(dest_path))
            s3destination.put(dest_path, clipped_item_upload)
            